## Final Project
You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

File descriptions
1. sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
2. test.csv - the test set. **You need to forecast the sales (total item count) for these shops and products for November 2015.**
3. sample_submission.csv - a sample submission file in the correct format.
4. items.csv - supplemental information about the items/products.
5. item_categories.csv  - supplemental information about the items categories.
6. shops.csv- supplemental information about the shops.

**Data fields**
-  ID - an Id that represents a (Shop, Item) tuple within the test set
-  shop_id - unique identifier of a shop
-  item_id - unique identifier of a product
-  item_category_id - unique identifier of item category
-  item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
-  item_price - current price of an item
-  date - date in format dd/mm/yyyy
-  date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
-  item_name - name of item
-  shop_name - name of shop
-  item_category_name - name of item category

### Modeling Strategy:
Use a holdout validation set, which is month 33. 
A BETTER strategy is to use cross-validation, which requires pipeline implementation.

In [2]:
import numpy as np
import pandas as pd 
import scipy.sparse 
import lightgbm 

for p in [np, pd, scipy, lightgbm]:
    print (p.__name__, p.__version__)

numpy 1.14.3
pandas 0.23.0
scipy 0.19.1
lightgbm 2.1.1


In [3]:
from itertools import product
from tqdm import tqdm_notebook
import gc 

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

### 1. Load the data and train/test split

#### 1.1 Load the data

In [14]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')
shops = pd.read_csv('../readonly/final_project_data/shops.csv')
items = pd.read_csv('../readonly/final_project_data/items.csv')
item_cats = pd.read_csv('../readonly/final_project_data/item_categories.csv')
sample_submission = pd.read_csv('../readonly/final_project_data/sample_submission.csv.gz')
sales_test = pd.read_csv('../readonly/final_project_data/test.csv.gz')

#### 1.2 Clean the data

In [15]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [16]:
# Remove outliers
sales = sales[sales['item_price'] < 100000]
sales = sales[sales['item_cnt_day'] < 1000]

The data are trasaction records. So for a day, shop, and an item, there are multiple records. The objective is to forecast monthly sales. The data is converted to monthly sales

In [17]:
index_col = ['date_block_num','shop_id','item_id']
sales = sales.groupby(index_col).agg({'item_cnt_day': np.sum, 'item_price': np.mean}).reset_index()
sales.rename({'item_cnt_day': 'item_cnt_month'}, axis='columns', inplace=True)

In [18]:
sales = pd.merge(sales, items, on='item_id',how='left')

In [19]:
sales = downcast_dtypes(sales)
sales = sales.drop(['item_name'], axis=1)

In [20]:
sales.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price,item_category_id
0,0,0,32,6.0,221.0,40
1,0,0,33,3.0,347.0,37
2,0,0,35,1.0,247.0,40
3,0,0,43,1.0,221.0,40
4,0,0,51,2.0,128.5,57


In [21]:
sales_test = pd.merge(sales_test, items, on='item_id',how='left')
sales_test = sales_test.drop(['item_name'], axis=1)
sales_test['date_block_num'] = 34
sales_test.head()

,ID,shop_id,item_id,item_category_id,date_block_num
0,0,5,5037,19,34
1,1,5,5320,55,34
2,2,5,5233,19,34
3,3,5,5232,23,34
4,4,5,5268,20,34


In [52]:
# Train/Test Split
train_df = sales.copy()
test_df = sales_test.copy()

### 2. Feature Extraction 

In [53]:
class Groupby_Avg_Featurizer(TransformerMixin):
    def __init__(self, group_col, value_col, output_col, include_negative=False, weight_col=None):
        self.group_col = group_col
        self.value_col = value_col
        self.weight_col = weight_col
        self.output_col = output_col
        self.include_negative=include_negative
        self.gb = None
        
    def fit(self, X):        
        
        if self.weight_col:
            if self.include_negative:
                X = X.loc[X[self.weight_col] > 0]
            self.gb = X.groupby(self.group_col).apply(lambda df: np.average(df[self.value_col])).reset_index().rename(columns={0:self.output_col})
        else:
            self.gb = X.groupby(self.group_col).apply(lambda df: np.average(df[self.value_col])).reset_index().rename(columns={0:self.output_col})
                      
        return self
    
    
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        assert isinstance(self.group_col, list)
        assert isinstance(self.value_col, str)
        if self.weight_col: assert isinstance(self.weight_col, str)
        assert isinstance(self.output_col, str)
        
                                                  
        return pd.merge(X, self.gb, on=self.group_col, how='left')

In [54]:
class Groupby_Std_Featurizer(TransformerMixin):
    def __init__(self, group_col, value_col, output_col, include_negative=False, weight_col=None):
        self.group_col = group_col
        self.value_col = value_col
        self.weight_col = weight_col
        self.output_col = output_col
        self.include_negative=include_negative
        self.gb = None
        
        
    def fit(self, X):        
        
            def weighted_std(df, values, weights):
                """
                Return the weighted average and standard deviation.
                values, weights -- column names
                """
                values, weights = df[values].values, df[weights].values
                average = np.average(values, weights=weights)
                variance = np.average((values-average)**2, weights=weights)
                return np.sqrt(variance)
        
            if self.weight_col:
                if self.include_negative:
                    X = X.loc[X[self.weight_col] > 0]
                self.gb = X.groupby(self.group_col).apply(weighted_std, self.value_col, self.weight_col).reset_index().rename(columns={0:self.output_col})
            else:
                self.gb = X.groupby(self.group_col).apply(lambda df: np.std(df[self.value_col])).reset_index().rename(columns={0:self.output_col})      
            return self
  
    def transform(self, X):
               
        assert isinstance(X, pd.DataFrame)
        assert isinstance(self.group_col, list)
        assert isinstance(self.value_col, str)
        if self.weight_col: assert isinstance(self.weight_col, str)
        assert isinstance(self.output_col, str)
        
        return pd.merge(X, self.gb, on=self.group_col, how='left')

In [55]:
class Groupby_Sum_Featurizer(TransformerMixin):
    '''
        Compute the weighted sum of two columns of a df
    '''
    def __init__(self, group_col, value_col, output_col, weight_col=None):
        self.group_col = group_col
        self.value_col = value_col
        self.weight_col = weight_col
        self.output_col = output_col
        self.gb = None
        
    def fit(self, X):        
        
        if self.weight_col:
            self.gb = X.groupby(self.group_col).apply(lambda df: np.dot(df[self.value_col].values, df[self.weight_col].values)).reset_index().rename(columns={0:self.output_col})
        else:
            self.gb = X.groupby(self.group_col).apply(lambda df: np.std(df[self.value_col])).reset_index().rename(columns={0:self.output_col})
        
        
        return self
    
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        assert isinstance(self.group_col, list)
        assert isinstance(self.value_col, str)
        if self.weight_col: assert isinstance(self.weight_col, str)
        assert isinstance(self.output_col, str)
             
        return pd.merge(X, self.gb, on=self.group_col, how='left')

In [56]:
class ClipTransformer(TransformerMixin):

    def __init__(self, a_min, a_max, col):
        self.a_min = a_min
        self.a_max = a_max
        self.col = col

    def fit(self, X, y=None):
        # stateless transformer
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        X.loc[:,self.col] = np.clip(X[self.col].values, self.a_min, self.a_max)
        return X

#### 2.1 Mean-encodings

In [58]:
# Not interacted with date_num_block because the lagged values captures the temporal effect

mean_encoding_pipeline = Pipeline(
[
    # item_count related features
    ('avg_all_shop_sales',             Groupby_Avg_Featurizer(['shop_id'], value_col='item_cnt_month',output_col='avg_all_shop_sales')),
    ('avg_all_item_sales',             Groupby_Avg_Featurizer(['item_id'], value_col='item_cnt_month',output_col='avg_all_item_sales')),
    ('avg_all_item_cat_sales',         Groupby_Avg_Featurizer(['item_category_id'], value_col='item_cnt_month',output_col='avg_all_item_cat_sales')),
    
    ('avg_shop_sales',          Groupby_Avg_Featurizer(['shop_id','date_block_num'], value_col='item_cnt_month',output_col='avg_shop_sales')),
    ('avg_item_sales',          Groupby_Avg_Featurizer(['item_id','date_block_num'], value_col='item_cnt_month',output_col='avg_item_sales')),
    ('avg_item_cat_sales',      Groupby_Avg_Featurizer(['item_category_id','date_block_num'], value_col='item_cnt_month',output_col='avg_item_cat_sales')),
    
    # price-related features
    ('avg_all_shop_price',             Groupby_Avg_Featurizer(['shop_id'], value_col='item_price',output_col='avg_all_shop_price', weight_col='item_cnt_month')),
    ('avg_all_item_price',             Groupby_Avg_Featurizer(['item_id'], value_col='item_price',output_col='avg_all_item_price', weight_col='item_cnt_month')),
    ('avg_all_item_cat_price',         Groupby_Avg_Featurizer(['item_category_id'], value_col='item_price',output_col='avg_all_item_cat_price', weight_col='item_cnt_month')),
    
    ('avg_shop_price',             Groupby_Avg_Featurizer(['shop_id','date_block_num'], value_col='item_price',output_col='avg_shop_price', weight_col='item_cnt_month')),
    ('avg_item_price',             Groupby_Avg_Featurizer(['item_id','date_block_num'], value_col='item_price',output_col='avg_item_price', weight_col='item_cnt_month')),
    ('avg_item_cat_price',         Groupby_Avg_Featurizer(['item_category_id','date_block_num'], value_col='item_price',output_col='avg_item_cat_price', weight_col='item_cnt_month')),
    
    
    # revenue related features
    ('all_shop_revenue',        Groupby_Sum_Featurizer(['shop_id'],value_col='item_price',output_col='all_shop_revenue', weight_col='item_cnt_month')),
    ('all_item_revenue',        Groupby_Sum_Featurizer(['item_id'],value_col='item_price',output_col='all_item_revenue', weight_col='item_cnt_month')),
    ('all_item_cat_revenue',    Groupby_Sum_Featurizer(['item_category_id'],value_col='item_price',output_col='all_item_cat_revenue', weight_col='item_cnt_month')),
    
    ('shop_revenue',        Groupby_Sum_Featurizer(['shop_id','date_block_num'],value_col='item_price',output_col='shop_revenue', weight_col='item_cnt_month')),
    ('item_revenue',        Groupby_Sum_Featurizer(['item_id','date_block_num'],value_col='item_price',output_col='item_revenue', weight_col='item_cnt_month')),
    ('item_cat_revenue',    Groupby_Sum_Featurizer(['item_category_id','date_block_num'],value_col='item_price',output_col='item_cat_revenue', weight_col='item_cnt_month'))
    
]
)
    

In [59]:
train_df = mean_encoding_pipeline.fit_transform(train_df)
train_df.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price,item_category_id,avg_all_shop_sales,avg_all_item_sales,avg_all_item_cat_sales,avg_shop_sales,...,avg_all_item_cat_price,avg_shop_price,avg_item_price,avg_item_cat_price,all_shop_revenue,all_item_revenue,all_item_cat_revenue,shop_revenue,item_revenue,item_cat_revenue
0,0,0,32,6.0,221.0,40,2.42792,2.664968,1.886919,2.338784,...,223.436295,442.008118,335.042328,213.911575,6636942.0,531776.750000,169960432.0,2966061.25,101512.085938,8667498.000
1,0,0,33,3.0,347.0,37,2.42792,1.487545,1.510428,2.338784,...,400.055206,442.008118,488.866669,456.995239,6636942.0,211944.500000,90525504.0,2966061.25,29831.000000,2843926.500
2,0,0,35,1.0,247.0,40,2.42792,2.960000,1.886919,2.338784,...,223.436295,442.008118,386.333344,213.911575,6636942.0,83343.046875,169960432.0,2966061.25,30210.000000,8667498.000
3,0,0,43,1.0,221.0,40,2.42792,1.000000,1.886919,2.338784,...,223.436295,442.008118,234.500000,213.911575,6636942.0,1505.000000,169960432.0,2966061.25,469.000000,8667498.000
4,0,0,51,2.0,128.5,57,2.42792,1.404762,1.340731,2.338784,...,281.264191,442.008118,233.937500,277.710052,6636942.0,62451.500000,9199066.0,2966061.25,2996.000000,427204.875


In [60]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609122 entries, 0 to 1609121
Data columns (total 24 columns):
date_block_num            1609122 non-null int32
shop_id                   1609122 non-null int32
item_id                   1609122 non-null int32
item_cnt_month            1609122 non-null float32
item_price                1609122 non-null float32
item_category_id          1609122 non-null int32
avg_all_shop_sales        1609122 non-null float64
avg_all_item_sales        1609122 non-null float64
avg_all_item_cat_sales    1609122 non-null float64
avg_shop_sales            1609122 non-null float64
avg_item_sales            1609122 non-null float64
avg_item_cat_sales        1609122 non-null float64
avg_all_shop_price        1609122 non-null float64
avg_all_item_price        1609122 non-null float64
avg_all_item_cat_price    1609122 non-null float64
avg_shop_price            1609122 non-null float64
avg_item_price            1609122 non-null float64
avg_item_cat_price        16

In [61]:
test_df = mean_encoding_pipeline.transform(test_df)
test_df.head()

,ID,shop_id,item_id,item_category_id,date_block_num,avg_all_shop_sales,avg_all_item_sales,avg_all_item_cat_sales,avg_shop_sales,avg_item_sales,...,avg_all_item_cat_price,avg_shop_price,avg_item_price,avg_item_cat_price,all_shop_revenue,all_item_revenue,all_item_cat_revenue,shop_revenue,item_revenue,item_cat_revenue
0,0,5,5037,19,34,1.773768,2.873303,2.648287,NaN,NaN,...,1423.005371,NaN,NaN,NaN,38229020.0,2.424548e+06,413319424.0,NaN,NaN,NaN
1,1,5,5320,55,34,1.773768,NaN,1.404199,NaN,NaN,...,281.804749,NaN,NaN,NaN,38229020.0,NaN,100022624.0,NaN,NaN,NaN
2,2,5,5233,19,34,1.773768,2.668421,2.648287,NaN,NaN,...,1423.005371,NaN,NaN,NaN,38229020.0,4.006051e+05,413319424.0,NaN,NaN,NaN
3,3,5,5232,23,34,1.773768,1.855263,2.369020,NaN,NaN,...,1453.872437,NaN,NaN,NaN,38229020.0,1.083592e+05,260314720.0,NaN,NaN,NaN
4,4,5,5268,20,34,1.773768,NaN,4.659603,NaN,NaN,...,2708.510254,NaN,NaN,NaN,38229020.0,NaN,374149664.0,NaN,NaN,NaN


In [62]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 0 to 214199
Data columns (total 23 columns):
ID                        214200 non-null int64
shop_id                   214200 non-null int64
item_id                   214200 non-null int64
item_category_id          214200 non-null int64
date_block_num            214200 non-null int64
avg_all_shop_sales        214200 non-null float64
avg_all_item_sales        198954 non-null float64
avg_all_item_cat_sales    214200 non-null float64
avg_shop_sales            0 non-null float64
avg_item_sales            0 non-null float64
avg_item_cat_sales        0 non-null float64
avg_all_shop_price        214200 non-null float64
avg_all_item_price        198954 non-null float64
avg_all_item_cat_price    214200 non-null float64
avg_shop_price            0 non-null float64
avg_item_price            0 non-null float64
avg_item_cat_price        0 non-null float64
all_shop_revenue          214200 non-null float64
all_item_revenue          198

#### Imputation
Some items in the test dataset do not exist in train dataset. Imputation is needed. 

In [32]:
for df in train_df, test_df:
    for feat in df.columns[4:]:
        if 'sales' in feat:
            df[feat]=df[feat].fillna(0)            # never been sold
        else:
            df[feat]=df[feat].fillna(df[feat].median())           # median price or revenue

In [33]:
test_df.info()         # All non-null values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 0 to 214199
Data columns (total 14 columns):
ID                    214200 non-null int64
shop_id               214200 non-null int64
item_id               214200 non-null int64
item_category_id      214200 non-null int64
date_block_num        214200 non-null int64
avg_shop_sales        214200 non-null float64
avg_item_sales        214200 non-null float64
avg_item_cat_sales    214200 non-null float64
avg_shop_price        214200 non-null float64
avg_item_price        214200 non-null float64
avg_item_cat_price    214200 non-null float64
shop_revenue          214200 non-null float64
item_revenue          214200 non-null float64
item_cat_revenue      214200 non-null float64
dtypes: float64(9), int64(5)
memory usage: 24.5 MB


#### 2.2 Lag features

In [34]:
def add_lag_data(df, data, features, periods, index_col, time_col):
    '''
        data: the input dataframe to get the lagged values from
        index_col: the index columns to join on
    
    '''
    
    assert isinstance(time_col, str)
    assert isinstance(index_col, list)
    
    
    for p in periods:
        data_copy = data.copy()         
        data_copy[time_col] += p    
        data_copy = data_copy[[time_col] +  index_col + features]        
        data_copy = data_copy.drop_duplicates(subset=[time_col] + index_col)
        data_copy.rename({
            feat: feat+"_"+'lag_'+str(p) for feat in features
        }, axis=1, inplace=True)
        df = pd.merge(df, data_copy, on=[time_col] + index_col, how='left')
    return df

In [35]:
lag_features_by_shop_item = ['item_cnt_month']
lag_features_by_item = [                    
                        'avg_item_sales',
                        'avg_item_cat_sales',
                        'avg_item_price',
                        'avg_item_cat_price',
                        'item_revenue',
                        'item_cat_revenue' 
                        ]
lag_features_by_shop = [
                        'avg_shop_sales',
                        'avg_shop_price',
                        'shop_revenue'   
                        ]

index_col_by_shop_item = ['shop_id','item_id']
index_col_by_item = ['item_id']
index_col_by_shop = ['shop_id']

lag_periods = [1,2,3,4,6,12]

In [36]:
# DEBUG CODE
# join = train_df.copy()
# join['date_block_num'] += 1
# join = join[['date_block_num', 'item_id', 'shop_id'] + lag_features]  
# join.rename({
#             feat: feat+"_"+'lag_'+str(p) for feat in lag_features
#         }, axis=1, inplace=True)
# test_join = pd.merge(test_df, join, on=['shop_id', 'item_id', 'date_block_num'], how='left')

In [41]:
# train_df = add_lag_data(train_df, train_df, lag_features_by_shop_item, lag_periods)
train_df = add_lag_data(train_df, train_df, lag_features_by_shop, lag_periods, index_col_by_shop, 'date_block_num')


# test_df = add_lag_data(test_df, train_df, lag_features, lag_periods)

In [51]:
cols = [col for col in train_df.columns if 'shop_sales_lag' in col]
train_df.loc[~train_df.avg_shop_sales_lag_12.isnull(), cols]

,avg_shop_sales_lag_1,avg_shop_sales_lag_2,avg_shop_sales_lag_3,avg_shop_sales_lag_4,avg_shop_sales_lag_6,avg_shop_sales_lag_12
687723,1.968752,1.968752,1.968752,1.968752,1.968752,1.968752
687724,1.968752,1.968752,1.968752,1.968752,1.968752,1.968752
687725,1.968752,1.968752,1.968752,1.968752,1.968752,1.968752
687726,1.968752,1.968752,1.968752,1.968752,1.968752,1.968752
687727,1.968752,1.968752,1.968752,1.968752,1.968752,1.968752
687728,1.968752,1.968752,1.968752,1.968752,1.968752,1.968752
687729,1.968752,1.968752,1.968752,1.968752,1.968752,1.968752
687730,1.968752,1.968752,1.968752,1.968752,1.968752,1.968752
687731,1.968752,1.968752,1.968752,1.968752,1.968752,1.968752
687732,1.968752,1.968752,1.968752,1.968752,1.968752,1.968752


In [38]:
train_df.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price,item_category_id,avg_shop_sales,avg_item_sales,avg_item_cat_sales,avg_shop_price,...,avg_item_price_lag_6,avg_item_cat_price_lag_6,item_revenue_lag_6,item_cat_revenue_lag_6,avg_item_sales_lag_12,avg_item_cat_sales_lag_12,avg_item_price_lag_12,avg_item_cat_price_lag_12,item_revenue_lag_12,item_cat_revenue_lag_12
0,0,0,32,6.0,221.0,40,2.42792,2.664968,1.886919,461.618958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,33,3.0,347.0,37,2.42792,1.487545,1.510428,461.618958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,35,1.0,247.0,40,2.42792,2.960000,1.886919,461.618958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,43,1.0,221.0,40,2.42792,1.000000,1.886919,461.618958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,51,2.0,128.5,57,2.42792,1.404762,1.340731,461.618958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
train_df = downcast_dtypes(train_df)

In [147]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609122 entries, 0 to 1609121
Data columns (total 75 columns):
date_block_num               1609122 non-null int32
shop_id                      1609122 non-null int32
item_id                      1609122 non-null int32
item_cnt_month               1609122 non-null float32
item_price                   1609122 non-null float32
item_category_id             1609122 non-null int32
avg_shop_sales               1609122 non-null float32
avg_item_sales               1609122 non-null float32
avg_item_cat_sales           1609122 non-null float32
avg_shop_price               1609122 non-null float32
avg_item_price               1609122 non-null float32
avg_item_cat_price           1609122 non-null float32
shop_revenue                 1609122 non-null float32
item_revenue                 1609122 non-null float32
item_cat_revenue             1609122 non-null float32
item_cnt_month_lag_1         676737 non-null float32
avg_shop_sales_lag_1         676

In [164]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 0 to 214199
Data columns (total 74 columns):
ID                           214200 non-null int64
shop_id                      214200 non-null int64
item_id                      214200 non-null int64
item_category_id             214200 non-null int64
date_block_num               214200 non-null int64
avg_shop_sales               214200 non-null float64
avg_item_sales               214200 non-null float64
avg_item_cat_sales           214200 non-null float64
avg_shop_price               214200 non-null float64
avg_item_price               214200 non-null float64
avg_item_cat_price           214200 non-null float64
shop_revenue                 214200 non-null float64
item_revenue                 214200 non-null float64
item_cat_revenue             214200 non-null float64
item_cnt_month_lag_1         28680 non-null float32
avg_shop_sales_lag_1         28680 non-null float32
avg_item_sales_lag_1         28680 non-null float32
a

In [166]:
test_df[test_df.avg_shop_sales_lag_1.isnull()].head()

,ID,shop_id,item_id,item_category_id,date_block_num,avg_shop_sales,avg_item_sales,avg_item_cat_sales,avg_shop_price,avg_item_price,...,item_cnt_month_lag_12,avg_shop_sales_lag_12,avg_item_sales_lag_12,avg_item_cat_sales_lag_12,avg_shop_price_lag_12,avg_item_price_lag_12,avg_item_cat_price_lag_12,shop_revenue_lag_12,item_revenue_lag_12,item_cat_revenue_lag_12
0,0,5,5037,19,34,1.773768,2.873303,2.648287,804.75824,1960.580566,...,1.0,1.773768,2.873303,2.648287,804.75824,1960.580566,1423.005371,38229020.0,2.424548e+06,413319424.0
1,1,5,5320,55,34,1.773768,0.000000,1.404199,804.75824,488.362488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5,5232,23,34,1.773768,1.855263,2.369020,804.75824,792.527710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,5268,20,34,1.773768,0.000000,4.659603,804.75824,488.362488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,5,5046,55,34,1.773768,2.086792,1.404199,804.75824,326.776245,...,1.0,1.773768,2.086792,1.404199,804.75824,326.776245,281.804749,38229020.0,1.782825e+05,100022624.0


In [170]:
sales[(sales.shop_id == 5) & (sales.date_block_num == 33)]

,date_block_num,shop_id,item_id,item_cnt_month,item_price,item_category_id
1578900,33,5,33,1.0,199.0,37
1578901,33,5,969,1.0,198.0,37
1578902,33,5,1121,2.0,349.0,55
1578903,33,5,1307,2.0,349.0,55
1578904,33,5,1384,2.0,198.0,30
1578905,33,5,1388,1.0,1499.0,19
1578906,33,5,1391,3.0,898.0,24
1578907,33,5,1465,1.0,999.0,19
1578908,33,5,1466,1.0,999.0,23
1578909,33,5,1469,1.0,1199.0,19


In [83]:
train_df = train_df.loc[train_df.date_block_num >= 12]

In [84]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889868 entries, 687723 to 1577590
Data columns (total 60 columns):
date_block_num                       889868 non-null int32
shop_id                              889868 non-null int32
item_id                              889868 non-null int32
item_cnt_month                       889868 non-null float32
item_price                           889868 non-null float32
item_category_id                     889868 non-null int32
avg_shop_sales                       889868 non-null float64
avg_monthly_shop_sales               889868 non-null float64
avg_item_sales                       889868 non-null float64
avg_monthly_item_sales               889868 non-null float64
avg_item_cat_sales                   889868 non-null float64
avg_monthly_item_cat_sales           889868 non-null float64
avg_shop_price                       889868 non-null float64
avg_item_price                       889868 non-null float64
avg_item_cat_price                   

In [91]:
train_df[train_df.avg_monthly_shop_sales_lag_1.isnull()].head(10)

,date_block_num,shop_id,item_id,item_cnt_month,item_price,item_category_id,avg_shop_sales,avg_monthly_shop_sales,avg_item_sales,avg_monthly_item_sales,...,monthly_shop_revenue_lag_6,monthly_item_revenue_lag_6,monthly_item_cat_revenue_lag_6,item_cnt_month_lag_12,avg_monthly_shop_sales_lag_12,avg_monthly_item_sales_lag_12,avg_monthly_item_cat_sales_lag_12,monthly_shop_revenue_lag_12,monthly_item_revenue_lag_12,monthly_item_cat_revenue_lag_12
687723,12,2,32,1.0,119.00,40,1.977966,1.84265,2.688312,2.709677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687725,12,2,99,1.0,149.00,37,1.977966,1.84265,1.182927,1.238095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687729,12,2,839,1.0,3300.00,73,1.977966,1.84265,2.224265,2.047619,...,863822.0,144900.000000,1.049163e+06,1.0,1.574176,3.523809,2.740891,1086138.0,234036.359375,1321520.75
687731,12,2,1406,1.0,999.00,28,1.977966,1.84265,3.704545,6.100000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687736,12,2,1415,1.0,1999.00,20,1.977966,1.84265,1.294118,1.222222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687738,12,2,1439,1.0,1000.00,32,1.977966,1.84265,2.997333,2.074074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687739,12,2,1441,1.0,500.00,32,1.977966,1.84265,3.838264,4.424242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687743,12,2,1471,2.0,1399.00,19,1.977966,1.84265,2.280324,1.935484,...,NaN,NaN,NaN,2.0,1.574176,6.697674,2.656906,1086138.0,741411.125000,14104524.00
687744,12,2,1480,7.0,499.00,30,1.977966,1.84265,3.371345,9.900000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687755,12,2,1832,2.0,899.25,19,1.977966,1.84265,2.933333,2.000000,...,863822.0,80503.898438,1.155088e+07,1.0,1.574176,1.600000,2.656906,1086138.0,63962.359375,14104524.00


In [92]:
train_df[(train_df.date_block_num < 12) & (train_df.shop_id == 2)]

,date_block_num,shop_id,item_id,item_cnt_month,item_price,item_category_id,avg_shop_sales,avg_monthly_shop_sales,avg_item_sales,avg_monthly_item_sales,...,monthly_shop_revenue_lag_6,monthly_item_revenue_lag_6,monthly_item_cat_revenue_lag_6,item_cnt_month_lag_12,avg_monthly_shop_sales_lag_12,avg_monthly_item_sales_lag_12,avg_monthly_item_cat_sales_lag_12,monthly_shop_revenue_lag_12,monthly_item_revenue_lag_12,monthly_item_cat_revenue_lag_12


In [104]:
sales[(sales.date_block_num == 11) & (sales.shop_id == 2)]

,date_block_num,shop_id,item_id,item_cnt_month,item_price,item_category_id
621448,11,2,33,1.0,199.00,37
621449,11,2,482,1.0,3300.00,73
621450,11,2,485,1.0,300.00,73
621451,11,2,791,1.0,600.00,73
621452,11,2,804,1.0,240.00,49
621453,11,2,806,1.0,249.00,49
621454,11,2,1007,1.0,599.00,67
621455,11,2,1015,1.0,449.00,67
621456,11,2,1031,1.0,28.00,67
621457,11,2,1059,1.0,449.00,67


In [101]:
sales.loc[sales.date_block_num == 11,'shop_id'].unique() == sales.loc[sales.date_block_num == 12,'shop_id'].unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [100]:
sales.loc[sales.date_block_num == 12,'shop_id'].unique()

array([ 2,  3,  4,  5,  6,  7, 10, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22,
       24, 25, 26, 27, 28, 29, 30, 31, 35, 37, 38, 41, 42, 43, 44, 45, 46,
       47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], dtype=int64)

In [97]:
sales.groupby('date_block_num')['shop_id'].nunique()

date_block_num
0     45
1     46
2     46
3     46
4     45
5     46
6     46
7     45
8     45
9     46
10    45
11    46
12    46
13    46
14    48
15    49
16    49
17    49
18    50
19    51
20    50
21    52
22    50
23    50
24    50
25    47
26    46
27    47
28    44
29    43
30    43
31    42
32    43
33    44
Name: shop_id, dtype: int64

In [9]:
a = pd.DataFrame({'key':[1,2,3,4],'value':[10,20,30,40]})
b = pd.DataFrame({'key':[2,2,4,4],'value':[200,210,400,410]})

In [10]:
a

,key,value
0,1,10
1,2,20
2,3,30
3,4,40


In [11]:
b

,key,value
0,2,200
1,2,210
2,4,400
3,4,410


In [13]:
c = pd.merge(a, b, on='key',how='inner')
c

,key,value_x,value_y
0,2,20,200
1,2,20,210
2,4,40,400
3,4,40,410
